In [20]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import math
from networkx.algorithms import approximation as approx

In [236]:
class effectiveResistanceOpt():
    def __init__(self):
        self.eps, self.currentStep, self.maxIterationTime = 10**-6, 0, 50
        self.MaximumStep = 50
        self.solved = False
        
    def initializeGraph(self):
        self.graph = nx.DiGraph()
        data = [(0,1,2),(1,2,1),(2,3,1),(3,0,1)]
        self.graph.add_weighted_edges_from(data)
        self.nNode = self.graph.number_of_nodes()
        self.nEdge = nx.number_of_edges(self.graph)
        self.incidenceMatrix = nx.incidence_matrix(self.graph, oriented=True).todense()
        self.incidenceMatrixUndirected = nx.incidence_matrix(self.graph).todense()
    
    def initializeWeights(self,data = [0.2, 0.3, 0.2, 0.3]):
        self.w = np.array(data)
        self.w_ = np.diag(self.w)
        self.w, self.w_ = np.asmatrix(self.w).T, np.asmatrix(self.w_)
        for i, edge in enumerate(self.graph.edges):
            self.graph.edges[edge]['weight'] = self.w[i,0]
    
    def addConstrs(self):
        self.constrsCoeff = np.ones((self.nEdge,1))
    
    def iMatrix(self):
        return self.incidenceMatrix
        
    def lMatrix(self,weight):
        w_ = np.asmatrix(np.diagflat(weight))
        laplacianMatrix = self.incidenceMatrix * w_ * self.incidenceMatrix.T
        return laplacianMatrix
    
    def computeObj(self,weight):
        x = self.computeX(weight)
        totalEffectiveResistace = (self.nNode * np.matrix.trace(x) - self.nNode)[0,0]
        return totalEffectiveResistace
    
    def computeX(self,w):
        lMatrix = self.lMatrix(w)
        x = np.linalg.inv(lMatrix + (1/self.nNode) * np.dot(np.ones((self.nNode,1)), np.ones((1, self.nNode))))
        return x
    def computeGradient(self,w):
        x = self.computeX(w)
        g = - self.nNode * (self.incidenceMatrix.T * x * x * self.incidenceMatrix).diagonal()
        return g
    
    def computeHessian(self,w):
        x = self.computeX(w)
        h = 2 * self.nNode * np.multiply((self.incidenceMatrix.T *x * x * self.incidenceMatrix),(self.incidenceMatrix.T * x * self.incidenceMatrix))
        return h
    
    def computeDw(self,w):
        h = self.computeHessian(w)
        g = self.computeGradient(w)
        upper = np.concatenate((h,self.constrsCoeff),1)
        lower = np.asmatrix(np.append(self.constrsCoeff,0))
        tempg = np.asmatrix(np.append(np.array(g),0)).T
        tempw = np.linalg.inv(np.concatenate((upper,lower),0)) * (-tempg)
        dw = tempw[0:self.nEdge,0]
        return dw
    
    def computeReducedcost(self,w):
        h = self.computeHessian(w)
        dw = self.computeDw(w)
        reducedcost = (dw.T * h * dw)[0,0]**0.5
        return reducedcost
    
    def NotOptimal(self,w):
        return self.computeReducedcost(w) >= self.eps

    def solve(self):
        iter = 0
        w = self.w
        while (self.NotOptimal(w)) and (iter < self.MaximumStep):
            alpha,beta,t = 0.3,0.5,1
            dw = self.computeDw(w)
            oldObj = self.computeObj(w)
            g = self.computeGradient(w)
            decrease = (g * dw)[0,0]
            while (self.computeObj(w + t * dw) > oldObj + alpha * t * decrease):
                t = beta * t
            iter += 1
            w += t * dw
            print("iter",iter)
            print("w:",w)

In [237]:
effectiveResistanceOpt_ = effectiveResistanceOpt()
effectiveResistanceOpt_.initializeGraph()
effectiveResistanceOpt_.addConstrs()
effectiveResistanceOpt_.initializeWeights()

In [238]:
effectiveResistanceOpt_.solve()

iter 1
w: [[0.24285714]
 [0.25714286]
 [0.24285714]
 [0.25714286]]
iter 2
w: [[0.24997673]
 [0.25002327]
 [0.24997673]
 [0.25002327]]
iter 3
w: [[0.25]
 [0.25]
 [0.25]
 [0.25]]
